In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
def create_windows_np(data, window_size, stride):
    num_samples, num_channels = data.shape
    num_windows = (num_samples - window_size) // stride + 1

    shape = (num_windows, window_size, num_channels)
    strides = (data.strides[0] * stride, data.strides[0], data.strides[1])

    windows = np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

    # transpose the windows array to the desired shape
    windows = np.transpose(windows, axes=(0, 2, 1))

    return windows

In [8]:
def noise_transform_vectorized(X, sigma=0.05):
    """
    Adding random Gaussian noise with mean 0
    """
    noise = np.random.normal(loc=0, scale=sigma, size=X.shape)
    return X + noise

def scaling_transform_vectorized(X, sigma=0.1):
    """
    Scaling by a random factor
    """
    scaling_factor = np.random.normal(loc=1.0, scale=sigma, size=(X.shape[0], 1, X.shape[2]))
    return X * scaling_factor

def negate_transform_vectorized(X):
    """
    Inverting the signals
    """
    return X * -1

def time_flip_transform_vectorized(X):
    """
    Reversing the direction of time
    """
    return X[:, ::-1, :]

In [9]:
# Define the list of transformations to be applied
transformations = [
    lambda x: noise_transform_vectorized(x), 
    lambda x: scaling_transform_vectorized(x),
    lambda x: negate_transform_vectorized(x),
    lambda x: time_flip_transform_vectorized(x),
]

In [10]:
def add_transformations(df):

    user_data = create_windows_np(df.loc[:, ['x', 'y', 'z']].values.astype(np.float32), 100, 50)

    # Get the number of windows and window size for the user's data
    num_windows, _, _ = user_data.shape

    # Apply the transformations to the user's data
    transformed_data = np.concatenate([transform_fn(user_data) for transform_fn in transformations], axis=0)
    transformed_data = np.concatenate([transformed_data, user_data], axis=0)
    transformed_data = np.array(transformed_data)

    # Create the labels for the transformed data
    transformed_labels = np.array([False for _ in range(4)])
    transformed_labels = np.append(transformed_labels, True)
    transformed_labels = np.repeat(transformed_labels, num_windows)

    return {
        'X': transformed_data,
        'y': transformed_labels
    }

In [11]:
class MultiTaskTPN(nn.Module):
    def __init__(self, num_tasks=len(transformations), num_channels=3):
        super(MultiTaskTPN, self).__init__()
        self.conv1 = nn.Conv1d(num_channels, 32, kernel_size=24, stride=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=16, stride=1)
        self.conv3 = nn.Conv1d(64, 96, kernel_size=8, stride=1)
        self.dropout = nn.Dropout(p=0.1)

        self.task_heads = nn.ModuleList([nn.Sequential(
            nn.Linear(96, 256),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256, 1),
            nn.Sigmoid()
        ) for _ in range(num_tasks)])

    def forward(self, x):
        x = self.dropout(nn.functional.relu(self.conv1(x)))
        x = self.dropout(nn.functional.relu(self.conv2(x)))
        x = self.dropout(nn.functional.relu(self.conv3(x)))
        x = nn.functional.max_pool1d(x, x.size(2)).squeeze(2)

        logits = [task_head(x).view(-1, 1) for task_head in self.task_heads]
        return logits


In [13]:
path = "../capture24"
batch_size = 256

# Create the model
model = MultiTaskTPN().to(device)

# Define the loss function and optimizer
loss_fn = nn.BCELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    
    for i in range(9, 10):

        # ignore subjects with missing values
        if i == 7 or i == 15 or i == 16:
            continue

        df = pd.read_feather(f'{path}/training.feather', columns=['x', 'y', 'z', 'user_id'])

        # Get the data for the current user
        user_data = add_transformations(df.loc[df['user_id'] == i+1])

        # Create a DataLoader to iterate over the test data in batches
        train_dataset = TensorDataset(torch.tensor(user_data['X'], dtype=torch.float32), 
                                    torch.tensor(user_data['y'], dtype=torch.float32))
        
        # Create a DataLoader to iterate over the test data in batches
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        

        # Training loop
        for batch_idx, (data, labels) in enumerate(train_dataloader):
            data, labels = data.to(device), labels.to(device)

            optimizer.zero_grad()

            logits = model(data)

            losses = []

            # Calculate the loss for each task
            for logit in logits:
                losses.append(loss_fn(logit, labels.view(-1, 1)))
            
            # Sum the losses to get the total loss
            total_loss = sum(losses)
            
            total_loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print(f"Epoch: {epoch+1}/{num_epochs}, Subject: P{i+1:03d}, Batch: {batch_idx}, Loss: {total_loss.item()}")

        del df


KeyboardInterrupt: 

: 